In [ ]:
# ==========================================================
# 1. PREPARACIÓN Y MONTAJE
# ==========================================================
import os
import shutil
import torch
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

if not os.path.exists('/content/stylegan3'):
    !git clone https://github.com/NVlabs/stylegan3.git

!pip install ninja opensimplex


Mounted at /content/drive
Cloning into 'stylegan3'...
remote: Enumerating objects: 212, done.
remote: Counting objects: 100% (163/163), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 212 (delta 99), reused 90 (delta 90), pack-reused 49 (from 1)
Receiving objects: 100% (212/212), 4.16 MiB | 18.45 MiB/s, done.
Resolving deltas: 100% (108/108), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.0/268.0 kB 26.9 MB/s eta 0:00:00


In [ ]:
# ==========================================================
# 2. PARCHE DE LIBRERÍA (CORRECCIÓN DEFINITIVA DE BETAS)
# Modificamos la librería de PyTorch en el sistema
# ==========================================================
print("Aplicando parche preventivo en la librería PyTorch...")

adam_file = "/usr/local/lib/python3.12/dist-packages/torch/optim/adam.py"
if os.path.exists(adam_file):
    with open(adam_file, 'r') as f:
        content = f.read()

    # Buscamos la validación que lanza el error y la neutralizamos
    old_check = 'if not 0.0 <= betas[0] < 1.0:'
    new_check = 'betas = (float(betas[0]), float(betas[1]))\n        if not 0.0 <= betas[0] < 1.0:'

    if 'float(betas[0])' not in content:
        content = content.replace(old_check, new_check)
        with open(adam_file, 'w') as f:
            f.write(content)
        print("✅ Librería PyTorch parcheada con éxito.")

Aplicando parche preventivo en la librería PyTorch...
✅ Librería PyTorch parcheada con éxito.


In [ ]:
# ==========================================================
# 3. PARCHE DE OPS (MODO LENTO / COMPATIBILIDAD)
# ==========================================================
archivos_ops = [
    '/content/stylegan3/torch_utils/ops/bias_act.py',
    '/content/stylegan3/torch_utils/ops/upfirdn2d.py',
    '/content/stylegan3/torch_utils/ops/filtered_lrelu.py'
]
for ruta in archivos_ops:
    if os.path.exists(ruta):
        with open(ruta, 'r') as f:
            codigo = f.read()
        codigo = codigo.replace("if impl == 'cuda' and x.device.type == 'cuda' and _init():", "if False:")
        with open(ruta, 'w') as f:
            f.write(codigo)

In [ ]:

# ==========================================================
# 4. RUTAS Y LANZAMIENTO
# ==========================================================
drive_zip = "/content/drive/MyDrive/Recursos/Conjuntos_de_datos/dataset_526.zip"
#drive_pkl = "/content/drive/MyDrive/ModelosIA/StyleGAN3/stylegan3-t-ffhqu-256x256.pkl"
#drive_pkl = "/content/drive/MyDrive/training-runs-sg3/00048-stylegan3-t-dataset_final-gpus1-batch16-gamma2.8/network-snapshot-000100.pkl"
#drive_pkl = "/content/drive/MyDrive/ModelosIA/StyleGAN3/stylegan3-t-ffhqu-256x256.pkl"
local_zip = "/content/dataset_final.zip"
local_pkl = "/content/stylegan3_local.pkl"
OUT_DIR = "/content/drive/MyDrive/training-runs-sg3"

shutil.copyfile(drive_zip, local_zip)
shutil.copyfile(drive_pkl, local_pkl)
os.makedirs(OUT_DIR, exist_ok=True)

!python /content/stylegan3/train.py \
    --outdir={OUT_DIR} \
    --data={local_zip} \
    --resume={local_pkl} \
    --cfg=stylegan3-t \
    --gpus=1 \
    --batch=16 \
    --batch-gpu=4 \
    --gamma=2.8 \
    --cbase=16384 \
    --cmax=512 \
    --mirror=1 \
    --aug=ada \
    --snap=1 \
    --metrics=none \
    --kimg=5000


Training options:
{
  "G_kwargs": {
    "class_name": "training.networks_stylegan3.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "channel_base": 16384,
    "channel_max": 512,
    "magnitude_ema_beta": 0.9994456359721023
  },
  "D_kwargs": {
    "class_name": "training.networks_stylegan2.Discriminator",
    "block_kwargs": {
      "freeze_layers": 0
    },
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 16384,
    "channel_max": 512
  },
  "G_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.0025
  },
  "D_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.002
  },
  "loss_kwargs": {
    "class_name": "training.loss.StyleGAN2Loss",
    "r1_gamma": 2.8,
    "blur_init_sigma": 0
  },
  "data_loader_kwargs": {
    "pin_memory"